In [16]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import PorterStemmer
# Esto sirve para configurar NLTK. La primera vez puede tardar un poco
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')


[nltk_data] Downloading package punkt to /home/ozzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ozzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
df = pd.read_parquet('data/movies.pq')

In [3]:
df = df[df.release_year>'1990']
df = df[df.release_year<'2010']
df.shape

(16483, 21)

In [4]:
stemmer = PorterStemmer()
def content(row):

    cont = row.overview
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    cont=re.sub("[^a-zA-Z]"," ",str(cont))
    # Pasamos todo a minúsculas
    cont=cont.lower()
    # Tokenizamos para separar las palabras del cont
    cont=nltk.word_tokenize(cont)
    # Eliminamos las palabras de menos de 3 letras
    cont = [palabra for palabra in cont if len(palabra)>3]
    # Sacamos las Stopwords
    cont = [palabra for palabra in cont if not palabra in stopwords]





    #Keep the first 20 words
    cont= cont[:20]
    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    # Aplicamos la funcion para buscar la raiz de las palabras
    cont=[stemmer.stem(palabra) for palabra in cont]
    # Por ultimo volvemos a unir el cont
    cont= row.title.lower() + " " + " ".join(cont)

    return cont

df['content'] = df.apply(content, axis=1)

In [18]:
def content(row):

    cont = row.title + row.genres + row.director
    # Vamos a reemplzar los caracteres que no sean leras por espacios
    cont=re.sub("[^a-zA-Z]"," ",str(cont))
    # Pasamos todo a minúsculas
    cont=cont.lower()
    # Tokenizamos para separar las palabras del cont
    cont=nltk.word_tokenize(cont)
    # Eliminamos las palabras de menos de 3 letras
    cont = [palabra for palabra in cont if len(palabra)>3]
    # Sacamos las Stopwords
    cont = [palabra for palabra in cont if not palabra in stopwords]




    
    ## Hasta acá Normalizamos, ahora a stemmizar
    
    # Aplicamos la funcion para buscar la raiz de las palabras
    cont=[stemmer.stem(palabra) for palabra in cont]
    # Por ultimo volvemos a unir el cont
    cont=" ".join(cont)

    return cont

df['content'] = df.apply(content, axis=1)

In [19]:
#create the TF-IDF model
MAX_DF     = 0.95
MIN_DF     = 1
tfidf = TfidfVectorizer(#max_df=MAX_DF, min_df=MIN_DF, \
                        #max_features=10000,\
                        #ngram_range=(1,2),
                        #token_pattern = r"\b\w{5,}\b"
                        )

df['content'].dropna(inplace=True)
#tfidf_matrix = tfidf.fit_transform(content)
tfidf_matrix = tfidf.fit_transform(df.content.apply(lambda x: np.str_(x)))
#vecs=vec.fit_transform(data["imp"].apply(lambda x: np.str_(x)))
tfidf_matrix.shape


(45429, 17912)

In [15]:
cosine_similarities=cosine_similarity(tfidf_matrix) 

MemoryError: Unable to allocate 13.8 GiB for an array with shape (1854119085,) and data type float64

In [7]:
cosine_similarities.shape

(16483, 16483)

In [8]:
movie_id=data[data.Name==title]["ids"].values[0]
scores=list(enumerate(sim[movie_id]))
sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
sorted_scores=sorted_scores[1:]
movies=[data[movies[0]==data["ids"]]["Name"].values[0] for movies in sorted_scores]

NameError: name 'data' is not defined

In [10]:
df = pd.read_parquet('data/movies.pq')
cosine_similarity_scores = list(enumerate(cosine_similarities[0]))
cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)
print (cosine_similarity_scores[:5])

for i in cosine_similarity_scores[:6]:
    print (df.loc[i[0]].title, i[1])


[(0, 1.0), (1951, 0.5039454318478097), (5342, 0.29504903902119617), (14315, 0.23343223363567064), (3275, 0.22610385798286725)]
Toy Story 1.0
Hot Lead & Cold Feet 0.5039454318478097
The Crocodile Hunter: Collision Course 0.29504903902119617
The End of St. Petersburg 0.23343223363567064
JFK 0.22610385798286725
Identity 0.20230016610539722
